

### Generación de Modelos Predictivos

Este notebook se encarga de generar modelos predictivos.  Se puede correr en Google Colab o en el iMac.

Para usarlo en Google Colab, usar tipo_a_usar='GPU'
Para usarlo en el imac, usar tipo_a_usar='CPU'



In [1]:
tipo_a_usar='CPU'

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score,roc_auc_score, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb

ModuleNotFoundError: No module named 'xgboost'

In [65]:
if tipo_a_usar =='GPU':
  from google.colab import drive
  drive.mount('/content/drive')

Los datos de origen es el mismo sea desde Google Colab o desde el iMac.  La diferencia es que el primero usar referencia a Google Drive y el otro acceso directo desde el sistema operativo

In [66]:
if tipo_a_usar =='GPU':
  DATOS = pd.read_pickle('/content/drive/MyDrive//Innovaciones Tecnológicas Aplicadas/Universidad Autónoma de Occidente/GoogleColab/DATOS3_35.pkl')

if tipo_a_usar == 'CPU':
  DATOS = pd.read_pickle('/Users/jaimereinoso/DESARROLLO/UAOPiloto2/checkPoints/DATOS3_35.pkl')

# quitemos por ahora las columnas que no nos interesan
DATOS.drop(['id_estudiante','periodo','anno_grado_colegio','anno_fecha_admitido',
                    'colegio','ciudad_colegio','genero','periodo_fecha_fin','fecha_grado'], axis=1, inplace=True)

In [67]:
DATOS.shape

(1369, 2867)

In [51]:
# LABEL viene con 3 valores. 0:  los que no desertaron.  1:  los que desertaron, -1: los que no sabemos.  Estos ultimos son para hacer predicciones

DATOS['LABEL'].value_counts()

LABEL
 0    1065
-1     170
 1     134
Name: count, dtype: int64

In [52]:
DATOS.shape

(1369, 2867)

In [53]:
#!pip install imbalanced-learn
from imblearn.over_sampling import SMOTE

# solo tomamos en cuenta los 0 (no desertores) y 1 (desertores)
data = DATOS[DATOS['LABEL']>=0].copy() 

X = data.drop('LABEL', axis=1)
y = data['LABEL']

In [54]:
X.shape

(1199, 2866)

In [55]:


smote = SMOTE(sampling_strategy='auto')

# Aplica SMOTE para generar muestras sintéticas
X_resampled, y_resampled = smote.fit_resample(X, y)



In [57]:
X_resampled.shape

(2130, 2866)

In [56]:
y_resampled.value_counts()

LABEL
0    1065
1    1065
Name: count, dtype: int64

In [58]:
# Dividir los datos en conjuntos de entrenamiento y prueba

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2)
X_train.shape

(1704, 2866)

In [59]:

# Definir una lista de algoritmos que deseas probar
algorithms = [
   ('RandomForest',
    RandomForestClassifier(), {
        'n_estimators': [50, 100, 200,400],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10]
    },
    'CPU'),
    ('SVM', SVC(), {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf', 'poly']
    },
    'CPU'),
    ('XGBoost', xgb.XGBClassifier(), {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 4, 5],
        'tree_method':['hist']  # no aguantó gpu_hist en google_colab
    },
    'GPU')
]

In [60]:
# solo usar si se está en Google Colab para confirmar la asignación de una
# tarjeta gráfica

if tipo_a_usar == 'GPU':
  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
  else:
    print(gpu_info)

In [61]:
print(X_train.shape)
print(X_test.shape)

(1704, 2866)
(426, 2866)


In [62]:
import joblib

lista_de_modelos = []


for algo_name, model, param_grid, tipo in algorithms:

  if tipo == tipo_a_usar:

    modelo_a_salvar = []

    print('algo:', algo_name, 'model:',model, 'param grid:', param_grid)
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='roc_auc')
    grid_search.fit(X_train, y_train)

    # Evaluar el modelo
    y_pred = grid_search.best_estimator_.predict(X_test)

    # vamos a salvarlo en modelo_a_salvar
    modelo_a_salvar.append(algo_name)

    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    modelo_a_salvar.append(accuracy)
    modelo_a_salvar.append(roc_auc)
    modelo_a_salvar.append(conf_matrix)
    modelo_a_salvar.append(grid_search.best_estimator_)
    modelo_a_salvar.append(grid_search.best_params_)

    lista_de_modelos.append(modelo_a_salvar)

    joblib.dump(lista_de_modelos, 'lista_de_modelos.pkl')

    print("Algoritmo:", algo_name)
    print("Accuracy:", accuracy)
    print("Roc_Auc:", roc_auc)
    print("Conf_Matrix:", conf_matrix)


algo: RandomForest model: RandomForestClassifier() param grid: {'n_estimators': [50, 100, 200, 400], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
Algoritmo: RandomForest
Accuracy: 0.9413145539906104
Roc_Auc: 0.9415257048092869
Conf_Matrix: [[190  11]
 [ 14 211]]
algo: SVM model: SVC() param grid: {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf', 'poly']}
Algoritmo: SVM
Accuracy: 0.9154929577464789
Roc_Auc: 0.913101160862355
Conf_Matrix: [[175  26]
 [ 10 215]]


Si se está en Google Colab, al aplicar files.download el archivo queda en la carpeta DOWNLOAD del iMac.



In [46]:
import joblib


if tipo_a_usar =='GPU':
  from google.colab import files
  joblib.dump(lista_de_modelos,'lista_de_modelosGPU.pkl')
  files.download('lista_de_modelosGPU.pkl')
  # queda en download del iMac

else:
  joblib.dump(lista_de_modelos,'modelos/lista_de_modelosCPU.pkl')
  print('entro aqui')
  # queda en el directorio de UAOPiloto2


entro aqui
